In [1]:
from bs4 import BeautifulSoup 
import requests 

def get_soup_by_link(link):
    response = requests.get(link) 
    return BeautifulSoup(response.text, 'html.parser')
 
def parse_rating(table_movie_rating): 
    try: 
        return table_movie_rating.tbody.tr.td.div.span.text 
    except: 
        return "Unknown" 
    
def parse_genre(table_movie_info): 
    try: 
        return table_movie_info.find('td', {'class': 'genre'}).p.a.text
    except: 
        return "Unknown" 
    
def parse_year(table_movie_info):
    try:
        return table_movie_info.find('td', {'class': 'year'}).a.text
    except:
        return "Unknown"

def parse(table_movie_info, class_name):
    try:
        return table_movie_info.find('td', {'class': class_name}).text
    except:
        return "Unknown"

def parse_min_age(td_with_data):
    try:
        return td_with_data.find('div', {'class': 'title__labels'}).contents[1].text
    except:
        return "Unknown"

def parse_director(td_with_data):
    try:
        return td_with_data.findAll('p', recursive = False)[0].text[10:]
    except:
        return "Unknown"    

def parse_actors(td_with_data):
    try:
        return td_with_data.findAll('p', recursive = False)[1].text[9:]
    except:
        return "Unknown"

main_link = 'https://afisha.tut.by/film/'
soup = get_soup_by_link(main_link)
films_li = soup.findAll('li', {'class': 'lists__li'}) 
links = []
for film_li in films_li: 
    film_links = film_li.findAll('a', {'class': 'name'}) 
    if len(film_links) != 0: 
        link = film_links[0]['href']
        if link.startswith(main_link) and '?utm_source' not in link: 
            links.append(link)

movies = {}
links_amount = len(links)
i = 1
for link in links:
    print(str(i) + " of " + str(links_amount) + ": " + link)
    i = i + 1
    
    soup = get_soup_by_link(link)
    td_with_data = soup.find('td', {'class': 'post b-event-post'}) 
    table_movie_info = td_with_data.find('table', {'class': 'movie_info'}) 
    table_movie_rating = td_with_data.find('table', {'class': 'movie_rating'}) 
    name = td_with_data.h1.string
    
    movies[name] = {}
    movies[name]['link'] = link
    movies[name]['genre'] = parse_genre(table_movie_info)
    movies[name]['year'] = parse_year(table_movie_info)
    movies[name]['country'] = parse(table_movie_info, 'author')
    movies[name]['duration'] = parse(table_movie_info, 'duration')
    movies[name]['end_date'] = parse(table_movie_info, 'date')
    movies[name]['director'] = parse_director(td_with_data) 
    movies[name]['actors'] = parse_actors(td_with_data)
    movies[name]['rating'] = parse_rating(table_movie_rating)
    movies[name]['min_age'] = parse_min_age(td_with_data)

1 of 88: https://afisha.tut.by/film/ford-protiv-ferrari/
2 of 88: https://afisha.tut.by/film/angely-charli/
3 of 88: https://afisha.tut.by/film/upyri/
4 of 88: https://afisha.tut.by/film/miduey/
5 of 88: https://afisha.tut.by/film/khoroshiy-lzhec/
6 of 88: https://afisha.tut.by/film/strazhi-arktiki/
7 of 88: https://afisha.tut.by/film/portret-devushki-v-ogne/
8 of 88: https://afisha.tut.by/film/akvarel/
9 of 88: https://afisha.tut.by/film/ford_vs_ferrari_ru_sub/
10 of 88: https://afisha.tut.by/film/the_good_liar_ru_sub/
11 of 88: https://afisha.tut.by/film/korotkiy_metr_dramatic_pink/
12 of 88: https://afisha.tut.by/film/doktor-son/
13 of 88: https://afisha.tut.by/film/vo-vse-tyazhkoe/
14 of 88: https://afisha.tut.by/film/malefisenta-vladychica-tmy/
15 of 88: https://afisha.tut.by/film/tekst/
16 of 88: https://afisha.tut.by/film/semeyka-addams/
17 of 88: https://afisha.tut.by/film/terminator-temnye-sudby/
18 of 88: https://afisha.tut.by/film/joker-film/
19 of 88: https://afisha.tut.by/

In [2]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

spark = SparkSession.builder.appName("minald").getOrCreate()
sc = spark.sparkContext
pdDF = pd.DataFrame(movies).transpose()
pdDF

,link,genre,year,country,duration,end_date,director,actors,rating,min_age
Ford против Ferrari,https://afisha.tut.by/film/ford-protiv-ferrari/,Боевик,2019,"США, Франция",Unknown,В прокате до 24 ноября,Джеймс Мэнголд,"Кристиан Бэйл, Мэтт Дэймон, Катрина Балф, Джон...",Unknown,16+
Ангелы Чарли,https://afisha.tut.by/film/angely-charli/,Боевик,2019,США,119 минут,В прокате до 24 ноября,Элизабет Бэнкс,"Кристен Стюарт, Наоми Скотт, Элла Балинска, Сэ...",Unknown,16+
Упыри,https://afisha.tut.by/film/upyri/,Ужасы,2018,Беларусь,77 минут,В прокате до 24 ноября,Андрей Гринько,"Вадим Галыгин, Александр Старченко, Габриэль П...","7,9",18+
Мидуэй,https://afisha.tut.by/film/miduey/,Боевик,2019,"Китай, США",138 минут,В прокате до 24 ноября,Роланд Эммерих,"Мэнди Мур, Люк Эванс, Патрик Уилсон, Вуди Харр...","7,8",16+
Хороший лжец,https://afisha.tut.by/film/khoroshiy-lzhec/,Детектив,2019,США,Unknown,В прокате до 24 ноября,Хороший лжец,"Хелен Миррен, Джим Картер, Иэн Маккеллен, Расс...",Unknown,16+
...,...,...,...,...,...,...,...,...,...,...
TheatreHD: Хансард,https://afisha.tut.by/film/theatrehd-khansard/,Unknown,2019,Unknown,95 минут,В прокате до 17 декабря,Саймон Годвин,Unknown,Unknown,16+
Звездные Войны: Скайуокер. Восход,https://afisha.tut.by/film/zvezdnye-voyny-epiz...,Боевик,2019,США,Unknown,В прокате до 22 декабря,Джей Джей Абрамс,"Лупита Нионго, Марк Хэмилл, Оскар Айзек, Адам ...","10,0",16+
Star Wars: The Rise of Skywalker (RU SUB),https://afisha.tut.by/film/star_wars_the_rise_...,Боевик,2019,Unknown,165 минут,В прокате до 22 декабря,Unknown,Unknown,Unknown,Unknown
TheatreHD: Палладио. Власть архитектуры,https://afisha.tut.by/film/theatrehd-palladio-...,Документальный,2019,Италия,98 минут,В прокате до 22 декабря,Unknown,Unknown,Unknown,12+


In [5]:
movies_df = spark.createDataFrame(pdDF)
movies_df.show()
movies_df.createOrReplaceTempView("all_movies")
min_age_grouped = movies_df.groupBy("min_age").count()
min_age_grouped.show()

+--------------------+----------------+----+--------------------+----------+--------------------+--------------------+--------------------+-------+-------+
|                link|           genre|year|             country|  duration|            end_date|            director|              actors| rating|min_age|
+--------------------+----------------+----+--------------------+----------+--------------------+--------------------+--------------------+-------+-------+
|https://afisha.tu...|          Боевик|2019|        США, Франция|   Unknown|В прокате до 24 н...|      Джеймс Мэнголд|Кристиан Бэйл, Мэ...|Unknown|    16+|
|https://afisha.tu...|          Боевик|2019|                 США| 119 минут|В прокате до 24 н...|      Элизабет Бэнкс|Кристен Стюарт, Н...|Unknown|    16+|
|https://afisha.tu...|           Ужасы|2018|            Беларусь|  77 минут|В прокате до 24 н...|      Андрей Гринько|Вадим Галыгин, Ал...|    7,9|    18+|
|https://afisha.tu...|          Боевик|2019|          Китай, США

In [4]:
import pyspark.sql.functions as f

countries_from_rows = spark.sql("SELECT COUNTRY FROM all_movies")
countries = countries_from_rows.select(
  "COUNTRY",
  f.split("COUNTRY", ", ").alias("COUNTRY"),
  f.posexplode(f.split("COUNTRY", ", ")).alias("pos", "_COUNTRY")
  ).select("_COUNTRY")
countries.createOrReplaceTempView("countries")
country_counts = spark.sql("SELECT _COUNTRY, COUNT(*) FROM countries GROUP BY _COUNTRY ORDER BY _COUNTRY DESC")
country_counts.show()

actors_from_rows = spark.sql("SELECT actors FROM all_movies")
actors = actors_from_rows.select(
  "actors",
  f.split("actors", ", ").alias("actors"),
  f.posexplode(f.split("actors", ", ")).alias("pos", "_actors")
  ).select("_actors")
actors.createOrReplaceTempView("actors")
actors_counts = spark.sql("SELECT _actors, COUNT(*) FROM actors GROUP BY _actors ORDER BY COUNT(*) DESC")
actors_counts.show()

+----------------+--------+
|        _COUNTRY|count(1)|
+----------------+--------+
|          Япония|       3|
|         Эстония|       1|
|           Чехия|       1|
|         Франция|       4|
|             США|      23|
|          Россия|       9|
|Польша. Беларусь|       1|
|        Норвегия|       1|
|         Мексика|       1|
|           Ливан|       1|
|     Корея Южная|       2|
|           Китай|       5|
|          Канада|       4|
|          Италия|       2|
|       Испания. |       1|
|         Испания|       1|
|            Иран|       1|
|           Индия|       1|
|           Дания|       2|
|        Германия|       6|
+----------------+--------+
only showing top 20 rows

+-------------------+--------+
|            _actors|count(1)|
+-------------------+--------+
|            Unknown|      34|
|    Вуди Харрельсон|       2|
|          Эд Скрейн|       2|
|       Джон Бернтал|       2|
|           Зои Дойч|       2|
|        Оскар Айзек|       2|
|       Ханна Мюррэй|  

In [6]:
min_age_grouped.write.format("jdbc")\
    .option("driver", "org.postgresql.Driver")\
    .option("url", "jdbc:postgresql:postgres")\
    .option("dbtable", "min_age_grouped")\
    .option("user", "postgres")\
    .option("password", "postgres")\
    .save()

In [7]:
country_counts.write.format("jdbc")\
    .option("driver", "org.postgresql.Driver")\
    .option("url", "jdbc:postgresql:postgres")\
    .option("dbtable", "country_counts")\
    .option("user", "postgres")\
    .option("password", "postgres")\
    .save()

In [8]:
actors_counts.write.format("jdbc")\
    .option("driver", "org.postgresql.Driver")\
    .option("url", "jdbc:postgresql:postgres")\
    .option("dbtable", "actors_counts")\
    .option("user", "postgres")\
    .option("password", "postgres")\
    .save()